# Wokflow  **TEST** con Full Bayesiana

## Inicializacion

In [2]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


In [3]:
%%shell
mkdir -p "/content/.drive/My Drive/dmeyf"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/dmeyf" /content/buckets/b1


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets

R

# Base

In [ ]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-11-16 05:03:03 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,659095,35.2,1454485,77.7,1434117,76.6
Vcells,1223630,9.4,8388608,64.0,1975136,15.1


In [ ]:
plocal <- list()

# 501
plocal$qcanaritos <- 5L
plocal$min_data_in_leaf <- 2000L
plocal$learning_rate <- 1.0
plocal$gradient_bound <- 0.01


plocal$APO <- 1
plocal$ksemillerio <- 1


In [ ]:
PARAM <- list()
PARAM$experimento <- "Comp2_Entrega-003"
PARAM$semilla_primigenia <- 599983

In [ ]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

# Cargo Librerias

In [ ]:
require("data.table")
require("parallel")

if(!require("R.utils")) install.packages("R.utils", repos = "http://cran.us.r-project.org")
require("R.utils")

if( !require("primes") ) install.packages("primes", repos = "http://cran.us.r-project.org")
require("primes")

if( !require("utils") ) install.packages("utils", repos = "http://cran.us.r-project.org")
require("utils")

if( !require("rlist") ) install.packages("rlist", repos = "http://cran.us.r-project.org")
require("rlist")

if( !require("yaml")) install.packages("yaml", repos = "http://cran.us.r-project.org")
require("yaml")

if( !require("lightgbm") ) install.packages("lightgbm", repos = "http://cran.us.r-project.org")
require("lightgbm")

if( !require("DiceKriging") ) install.packages("DiceKriging", repos = "http://cran.us.r-project.org")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO", repos = "http://cran.us.r-project.org")
require("mlrMBO")

Loading required package: data.table

Loading required package: parallel

Loading required package: R.utils

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘R.utils’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’


Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successful

## Preprocesamiento

### Generacion de la clase_ternaria

In [ ]:
Sys.time()
#require( "data.table" )

# leo el dataset
#dataset <- fread("~/datasets/competencia_02_crudo.csv.gz" )
dataset <- fread("/content/buckets/b1/datasets/competencia_02_crudo.csv.gz", stringsAsFactors= TRUE )
# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

[1] "2025-11-16 05:13:00 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2427802,129.7,4009348,214.2,4009348,214.2
Vcells,725046045,5531.7,1062446325,8105.9,841575756,6420.8


[1] "2025-11-16 05:13:29 UTC"

In [ ]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
201901,BAJA+1,645
201901,BAJA+2,729
201901,CONTINUA,122899
201902,BAJA+1,733
201902,BAJA+2,707
201902,CONTINUA,123961
201903,BAJA+1,708
201903,BAJA+2,751
201903,CONTINUA,124508


### Eliminacion de Features

Completar a gusto LUEGO de realizar un analisis exploratorio de datos.
<br> No necesariamente en esta Segunda Competencia conviele eliminar los mismos campos que en la Primera ...

In [ ]:
# Salsa Magica para 202106
dataset[, mprestamos_personales := NULL ]
dataset[, cprestamos_personales := NULL ]

# Llevar a pesos y aplicar Ranking

In [ ]:
# Identify currency related columns in R and exclude 'foto_mes'
currency_cols_r <- colnames(dataset)[grepl("_m|^m", colnames(dataset))]
currency_cols_r <- setdiff(currency_cols_r, "foto_mes")
print("Currency related columns (excluding foto_mes):")
print(currency_cols_r)

[1] "Currency related columns (excluding foto_mes):"
 [1] "mrentabilidad"                       
 [2] "mrentabilidad_annual"                
 [3] "mcomisiones"                         
 [4] "mactivos_margen"                     
 [5] "mpasivos_margen"                     
 [6] "mcuenta_corriente_adicional"         
 [7] "mcuenta_corriente"                   
 [8] "mcaja_ahorro"                        
 [9] "mcaja_ahorro_adicional"              
[10] "mcaja_ahorro_dolares"                
[11] "mcuentas_saldo"                      
[12] "mautoservicio"                       
[13] "mtarjeta_visa_consumo"               
[14] "ctarjeta_master"                     
[15] "ctarjeta_master_transacciones"       
[16] "mtarjeta_master_consumo"             
[17] "mprestamos_prendarios"               
[18] "mprestamos_hipotecarios"             
[19] "mplazo_fijo_dolares"                 
[20] "mplazo_fijo_pesos"                   
[21] "minversion1_pesos"                   
[22] "minversion1_dolar

In [ ]:
# Identify dollar currency columns
dollar_cols_r <- currency_cols_r[grep("dolares|usd|dolares", currency_cols_r, ignore.case = TRUE)]

# Print the identified dollar columns
print("Dollar currency columns:")
print(dollar_cols_r)

[1] "Dollar currency columns:"
 [1] "mcaja_ahorro_dolares"    "mplazo_fijo_dolares"    
 [3] "minversion1_dolares"     "Master_msaldodolares"   
 [5] "Master_mconsumosdolares" "Master_madelantodolares"
 [7] "Master_mpagosdolares"    "Visa_msaldodolares"     
 [9] "Visa_mconsumosdolares"   "Visa_madelantodolares"  
[11] "Visa_mpagosdolares"     


In [ ]:
# Get unique foto_mes values from the dataset
unique_foto_mes <- sort(unique(dataset$foto_mes))

# Define approximate exchange rates for each month based on user input
exchange_rate_values <-  c(
  # 2019 (Ene - Dic)
  38.20, 40.10, 44.40, 45.80, 46.30, 43.70, 45.00, 61.00, 59.50, 63.50, 62.90, 63.00,

  # 2020 (Ene - Dic)
  63.00, 64.00, 66.50, 68.80, 70.60, 73.50, 76.10, 78.10, 80.25, 82.75, 85.75, 89.25,

  # 2021 (Ene - Ago)
  92.25, 94.50, 97.00, 98.50, 99.75, 101.25, 102.50, 103.50
) # User-provided rates

# Create the data.table with foto_mes and exchange_rate
exchange_rates <- data.table(
  foto_mes = unique_foto_mes,
  exchange_rate = exchange_rate_values
)

# Print the exchange_rates data.table
print(exchange_rates)

    foto_mes exchange_rate
       <int>         <num>
 1:   201901         38.20
 2:   201902         40.10
 3:   201903         44.40
 4:   201904         45.80
 5:   201905         46.30
 6:   201906         43.70
 7:   201907         45.00
 8:   201908         61.00
 9:   201909         59.50
10:   201910         63.50
11:   201911         62.90
12:   201912         63.00
13:   202001         63.00
14:   202002         64.00
15:   202003         66.50
16:   202004         68.80
17:   202005         70.60
18:   202006         73.50
19:   202007         76.10
20:   202008         78.10
21:   202009         80.25
22:   202010         82.75
23:   202011         85.75
24:   202012         89.25
25:   202101         92.25
26:   202102         94.50
27:   202103         97.00
28:   202104         98.50
29:   202105         99.75
30:   202106        101.25
31:   202107        102.50
32:   202108        103.50
    foto_mes exchange_rate


In [ ]:
# Iterate through dollar columns and convert to pesos
for (col_name in dollar_cols_r) {
  new_col_name <- paste0(col_name, "_pesos")

  # Merge with exchange rates temporarily based on foto_mes
  dataset <- merge(dataset, exchange_rates, by = "foto_mes", all.x = TRUE)

  # Calculate the converted value and assign to the new column
  dataset[, (new_col_name) := get(col_name) * exchange_rate]

  # Remove the temporary exchange_rate column
  dataset[, exchange_rate := NULL]
}

In [ ]:
# Select a few original dollar columns and their corresponding new peso columns
selected_dollar_cols <- c("mcaja_ahorro_dolares", "Master_msaldodolares", "Visa_mconsumosdolares")
converted_peso_cols <- paste0(selected_dollar_cols, "_pesos")
all_cols_to_display <- c("foto_mes", selected_dollar_cols, converted_peso_cols)

# Display a sample of these columns using print
print(dataset[, all_cols_to_display, with= FALSE])

Key: <foto_mes>
         foto_mes mcaja_ahorro_dolares Master_msaldodolares
            <int>                <num>                <num>
      1:   201901                 0.00                   NA
      2:   201901                 0.00                    0
      3:   201901                 0.00                    0
      4:   201901              2929.73                    0
      5:   201901               910.54                    0
     ---                                                   
4717954:   202108                 0.00                    0
4717955:   202108                 0.00                    0
4717956:   202108                 0.00                    0
4717957:   202108                 0.00                    0
4717958:   202108                 0.00                    0
         Visa_mconsumosdolares mcaja_ahorro_dolares_pesos
                         <num>                      <num>
      1:                    NA                       0.00
      2:                  0.00

In [ ]:
rank_percentiles <- function(x) {
  pos_indices <- which(x > 0)
  neg_indices <- which(x < 0)
  zero_indices <- which(x == 0)

  ranked_x <- numeric(length(x))

  if (length(pos_indices) > 0) {
    ranked_x[pos_indices] <- rank(x[pos_indices], ties.method = "average") / length(pos_indices)
  }

  if (length(neg_indices) > 0) {
    ranked_x[neg_indices] <- -rank(abs(x[neg_indices]), ties.method = "average") / length(neg_indices)
  }

  # Zero values remain 0
  # ranked_x[zero_indices] <- 0 # This is already handled by initialization

  return(ranked_x)
}

In [ ]:
# Apply the custom ranking function to the desired currency columns
cols_to_rank <- setdiff(currency_cols_r, dollar_cols_r)
cols_to_rank <- c(cols_to_rank, converted_peso_cols)


for (col_name in cols_to_rank) {
  new_col_name <- paste0(col_name, "_ranked")
  dataset[, (new_col_name) := rank_percentiles(get(col_name))]
}

In [ ]:
# Select a few original currency columns and their corresponding ranked columns
selected_cols <- c("mrentabilidad", "mrentabilidad_annual", "mcomisiones")
ranked_cols <- paste0(selected_cols, "_ranked")
all_cols_to_display <- c(selected_cols, ranked_cols)

# Display a sample of these columns using print
print(dataset[, all_cols_to_display, with= FALSE])

         mrentabilidad mrentabilidad_annual mcomisiones mrentabilidad_ranked
                 <num>                <num>       <num>                <num>
      1:        464.44             31108.76     1497.55           0.14320160
      2:        888.71              9787.31      848.09           0.26355982
      3:       1964.85             18571.31     1211.57           0.53695763
      4:        663.04             43899.63      649.36           0.19990426
      5:        732.51             36029.59      116.13           0.21963482
     ---                                                                    
4717954:         94.13                94.13        0.00           0.03365496
4717955:       2705.40              2705.40     2698.92           0.66053516
4717956:          0.77                 0.77        0.00           0.00170416
4717957:       2366.83              2366.83        0.00           0.61063879
4717958:          0.00                 0.00        0.00           0.00000000

In [ ]:
# Remove the original currency columns, dollar columns, and converted peso columns
cols_to_remove <- unique(c(currency_cols_r, dollar_cols_r, converted_peso_cols))
dataset <- dataset[, (cols_to_remove) := NULL]

# Print a sample of the remaining columns to verify
print(colnames(dataset))

  [1] "foto_mes"                                   
  [2] "numero_de_cliente"                          
  [3] "active_quarter"                             
  [4] "cliente_vip"                                
  [5] "internet"                                   
  [6] "cliente_edad"                               
  [7] "cliente_antiguedad"                         
  [8] "cproductos"                                 
  [9] "tcuentas"                                   
 [10] "ccuenta_corriente"                          
 [11] "ccaja_ahorro"                               
 [12] "cdescubierto_preacordado"                   
 [13] "ctarjeta_debito"                            
 [14] "ctarjeta_debito_transacciones"              
 [15] "ctarjeta_visa"                              
 [16] "ctarjeta_visa_transacciones"                
 [17] "cprestamos_prendarios"                      
 [18] "cprestamos_hipotecarios"                    
 [19] "cplazo_fijo"                                
 [20] "cinve

# -- Fin Ranking --

### Feature Engineering Intra-Mes

Crear variables nuevas a partir de las existentes dentro del mismo registro, **sin** ir a buscar información histórica.
<br> El siguiente código es un mínimo ejemplo, agregar nuevos features a gusto

In [ ]:
# el mes 1,2, ..12 , podria servir para detectar estacionalidad
dataset[, kmes := foto_mes %% 100]

# creo un ctr_quarter que tenga en cuenta cuando
# los clientes hace 3 menos meses que estan
# ya que seria injusto considerar las transacciones medidas en menor tiempo
dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter) ]
dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5.0]
dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2.0]
dataset[cliente_antiguedad == 3, ctrx_quarter_normalizado := ctrx_quarter * 1.2]

# variable extraida de una tesis de maestria de Irlanda, se perdió el link
dataset[, mpayroll_sobre_edad := mpayroll_ranked / cliente_edad]

Sys.time()

[1] "2025-11-16 05:18:57 UTC"

### Feature Engineering Historico

In [ ]:
if( !require("Rcpp")) install.packages("Rcpp", repos = "http://cran.us.r-project.org")
require("Rcpp")

Loading required package: Rcpp



In [ ]:
# se calculan para los 6 meses previos el minimo, maximo y
#  tendencia calculada con cuadrados minimos
# la formula de calculo de la tendencia puede verse en
#  https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/10%3A_Correlation_and_Regression/10.04%3A_The_Least_Squares_Regression_Line
# para la maxíma velocidad esta funcion esta escrita en lenguaje C,
# y no en la porqueria de R o Python

cppFunction("NumericVector fhistC(NumericVector pcolumna, IntegerVector pdesde )
{
  /* Aqui se cargan los valores para la regresion */
  double  x[100] ;
  double  y[100] ;

  int n = pcolumna.size();
  NumericVector out( 5*n );

  for(int i = 0; i < n; i++)
  {
    //lag
    if( pdesde[i]-1 < i )  out[ i + 4*n ]  =  pcolumna[i-1] ;
    else                   out[ i + 4*n ]  =  NA_REAL ;


    int  libre    = 0 ;
    int  xvalor   = 1 ;

    for( int j= pdesde[i]-1;  j<=i; j++ )
    {
       double a = pcolumna[j] ;

       if( !R_IsNA( a ) )
       {
          y[ libre ]= a ;
          x[ libre ]= xvalor ;
          libre++ ;
       }

       xvalor++ ;
    }

    /* Si hay al menos dos valores */
    if( libre > 1 )
    {
      double  xsum  = x[0] ;
      double  ysum  = y[0] ;
      double  xysum = xsum * ysum ;
      double  xxsum = xsum * xsum ;
      double  vmin  = y[0] ;
      double  vmax  = y[0] ;

      for( int h=1; h<libre; h++)
      {
        xsum  += x[h] ;
        ysum  += y[h] ;
        xysum += x[h]*y[h] ;
        xxsum += x[h]*x[h] ;

        if( y[h] < vmin )  vmin = y[h] ;
        if( y[h] > vmax )  vmax = y[h] ;
      }

      out[ i ]  =  (libre*xysum - xsum*ysum)/(libre*xxsum -xsum*xsum) ;
      out[ i + n ]    =  vmin ;
      out[ i + 2*n ]  =  vmax ;
      out[ i + 3*n ]  =  ysum / libre ;
    }
    else
    {
      out[ i       ]  =  NA_REAL ;
      out[ i + n   ]  =  NA_REAL ;
      out[ i + 2*n ]  =  NA_REAL ;
      out[ i + 3*n ]  =  NA_REAL ;
    }
  }

  return  out;
}")

In [ ]:
# calcula la tendencia de las variables cols de los ultimos 6 meses
# la tendencia es la pendiente de la recta que ajusta por cuadrados minimos
# La funcionalidad de ratioavg es autoria de  Daiana Sparta,  UAustral  2021

TendenciaYmuchomas <- function(
    dataset, cols, ventana = 6, tendencia = TRUE,
    minimo = TRUE, maximo = TRUE, promedio = TRUE,
    ratioavg = FALSE, ratiomax = FALSE) {
  gc(verbose= FALSE)
  # Esta es la cantidad de meses que utilizo para la historia
  ventana_regresion <- ventana

  last <- nrow(dataset)

  # creo el vector_desde que indica cada ventana
  # de esta forma se acelera el procesamiento ya que lo hago una sola vez
  vector_ids <- dataset[ , numero_de_cliente ]

  vector_desde <- seq(
    -ventana_regresion + 2,
    nrow(dataset) - ventana_regresion + 1
  )

  vector_desde[1:ventana_regresion] <- 1

  for (i in 2:last) {
    if (vector_ids[i - 1] != vector_ids[i]) {
      vector_desde[i] <- i
    }
  }
  for (i in 2:last) {
    if (vector_desde[i] < vector_desde[i - 1]) {
      vector_desde[i] <- vector_desde[i - 1]
    }
  }

  for (campo in cols) {
    nueva_col <- fhistC(dataset[, get(campo)], vector_desde)

    if (tendencia) {
      dataset[, paste0(campo, "_tend", ventana) :=
        nueva_col[(0 * last + 1):(1 * last)]]
    }

    if (minimo) {
      dataset[, paste0(campo, "_min", ventana) :=
        nueva_col[(1 * last + 1):(2 * last)]]
    }

    if (maximo) {
      dataset[, paste0(campo, "_max", ventana) :=
        nueva_col[(2 * last + 1):(3 * last)]]
    }

    if (promedio) {
      dataset[, paste0(campo, "_avg", ventana) :=
        nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratioavg) {
      dataset[, paste0(campo, "_ratioavg", ventana) :=
        get(campo) / nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratiomax) {
      dataset[, paste0(campo, "_ratiomax", ventana) :=
        get(campo) / nueva_col[(2 * last + 1):(3 * last)]]
    }
  }
}

In [ ]:
# Feature Engineering Historico
# Creacion de LAGs
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
  colnames(dataset),
  c("numero_de_cliente", "foto_mes", "clase_ternaria")
))

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
  paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# lags de orden 2
dataset[,
  paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
  dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
  dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}

Sys.time()

[1] "2025-11-16 05:20:57 UTC"

In [ ]:
# parametros de Feature Engineering Historico de Tendencias
PARAM$FE_hist$Tendencias$run <- TRUE
PARAM$FE_hist$Tendencias$ventana <- 6
PARAM$FE_hist$Tendencias$tendencia <- TRUE
PARAM$FE_hist$Tendencias$minimo <- TRUE
PARAM$FE_hist$Tendencias$maximo <- TRUE
PARAM$FE_hist$Tendencias$promedio <- TRUE
PARAM$FE_hist$Tendencias$ratioavg <- TRUE
PARAM$FE_hist$Tendencias$ratiomax <- TRUE

In [ ]:
# aqui se agregan las tendencias de los ultimos 6 meses

cols_lagueables <- intersect(cols_lagueables, colnames(dataset))
setorder(dataset, numero_de_cliente, foto_mes)

if( PARAM$FE_hist$Tendencias$run) {
    TendenciaYmuchomas(dataset,
    cols = cols_lagueables,
    ventana = PARAM$FE_hist$Tendencias$ventana, # 6 meses de historia
    tendencia = PARAM$FE_hist$Tendencias$tendencia,
    minimo = PARAM$FE_hist$Tendencias$minimo,
    maximo = PARAM$FE_hist$Tendencias$maximo,
    promedio = PARAM$FE_hist$Tendencias$promedio,
    ratioavg = PARAM$FE_hist$Tendencias$ratioavg,
    ratiomax = PARAM$FE_hist$Tendencias$ratiomax
  )
}

ncol(dataset)
Sys.time()

[1] 921

[1] "2025-11-16 05:21:57 UTC"

# Guardar datos Procesados.

In [ ]:
#fwrite( dataset,
#    file =  paste0("/content/buckets/b1/datasets/competencia_02_trabajado_",experimento_folder,".csv.gz"),
#    sep = ","
#)

In [ ]:
ncol(dataset)

[1] 921

# Levantar Datos Procesados.



In [ ]:
#dataset <- fread(paste0("/content/buckets/b1/datasets/competencia_02_trabajado_",experimento_folder,".csv.gz"), stringsAsFactors= TRUE)

In [ ]:
ncol(dataset)

[1] 921

## Modelado

No hay modelado, no se hace optimizacion de hiperparametros.

## Produccion

Las decisiones que se toman para la construccion del modelo final son:
* Los positvos son  POS={"BAJA+1", "BAJA+2"}, esta es una meticulosa decisión.
* Se entrena en los treinta meses del intervalo [201901, 202104]
* Se realiza undersampling al 5%
* Se utilizan los hiperparámetros optimos encontrados en la Bayesian Optimization
   * Se escala min_data_in_leaf

### Final Training Strategy

In [ ]:
PARAM$train_final$future <- c(202108)

PARAM$train_final$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106
)

PARAM$train_final$undersampling <- 0.1  # Silvana y Sofia

In [ ]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$training]

In [ ]:
# canaritos
PARAM$train_final$lgbm$qcanaritos <- plocal$qcanaritos

cols0 <- copy(colnames(dataset_train_final))
filas <- nrow(dataset_train_final)

if( PARAM$train_final$lgbm$qcanaritos > 0 ) {
  for( i in seq(PARAM$train_final$lgbm$qcanaritos) ){
    dataset_train_final[, paste0("canarito_",i) := runif( filas) ]
  }

  # las columnas canaritos mandatoriamente van al comienzo del dataset
  cols_canaritos <- copy( setdiff( colnames(dataset_train_final), cols0 ) )
  setcolorder( dataset_train_final, c( cols_canaritos, cols0 ) )
}

Sys.time()

[1] "2025-11-16 05:22:01 UTC"

#### Registros cambio las proporciones de POS/NEG

In [ ]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [ ]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

### Final Models

Aqui SIEMPRE voy a hacer un semillerio, independientemente de si en la Bayesian Optimization calculé un semillerio en cada iteración.
<br> Entreno un LightGBM para cada semilla,  y guardo el modelo dentro de la carpeta  **modelitos**
<br> Intencionalmente en una primera etapá se generan los modelos y graban, y en una segunda etapa se leen eso modelos y se aplican a los datos del futuro

APO controla cuantas veces se repite el modelo, que se usa para promediar ganancias y reportar en la Pseudo Competencia algo razonable
<br> El modelo puede ser un LightGBM simple (ksemillerio==1)  o un Ensemble Semillerio( ksemillerio > 1 )
<br> Lamentablmente APO necesita utilizar muchas semillas, y eso demanda TIEMPO de corrida

In [ ]:

PARAM$train_final$lgbm$param_completo <-  list(
  boosting= "gbdt",
  objective= "binary",
  metric= "custom",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE,
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_bin= 31L,
  min_data_in_leaf= plocal$min_data_in_leaf,  #este ya es el valor default de LightGBM

  num_iterations= 9999L, # dejo libre la cantidad de arboles, zLightGBM se detiene solo
  num_leaves= 9999L, # dejo libre la cantidad de hojas, zLightGBM sabe cuando no hacer un split
  learning_rate= plocal$learning_rate,  # se lo deja en 1.0 para que si el score esta por debajo de gradient_bound no se lo escale

  feature_fraction= 0.50, # un valor equilibrado, habra que probar alternativas ...

  canaritos= PARAM$train_final$lgbm$qcanaritos, # fundamental en zLightGBM, aqui esta el control del overfitting
  gradient_bound= plocal$gradient_bound   # default de zLightGBM
)

Sys.time()

[1] "2025-11-16 05:22:02 UTC"

In [ ]:
# Semillerio Final
PARAM$train_final$APO <- plocal$APO
PARAM$train_final$ksemillerio  <- plocal$ksemillerio

PARAM$train_final$cortes <- c(8000, 8500, 9000, 9500, 10000, 10500, 11000, 11500, 12000)

In [ ]:
# detach("package:lightgbm", unload= TRUE)

In [ ]:
if( !require("zlightgbm") ) install.packages("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/zlightgbm_4.6.0.99.tar.gz", repos= NULL, type= "source")
require("zlightgbm")

Loading required package: zlightgbm

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘zlightgbm’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: zlightgbm

Registered S3 methods overwritten by 'zlightgbm':
  method                 from    
  dimnames<-.lgb.Dataset lightgbm
  dim.lgb.Dataset        lightgbm
  dimnames.lgb.Dataset   lightgbm
  predict.lgb.Booster    lightgbm
  print.lgb.Booster      lightgbm
  summary.lgb.Booster    lightgbm


Attaching package: ‘zlightgbm’


The following objects are masked from ‘package:lightgbm’:

    get_field, getLGBMthreads, lgb.configure_fast_predict,
    lgb.convert_with_rules, lgb.cv, lgb.Dataset, lgb.Dataset.construct,
    lgb.Dataset.create.valid, lgb.Dataset.save,
    lgb.Dataset.set.categorical, lgb.Dataset.set.reference,
    lgb.drop_serialized, lgb.dump, lgb.get.eval.result, lgb.importance,
    lg

In [ ]:
if(!require("primes")) install.packages("primes")
require("primes")

In [ ]:
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
PARAM$train_final$semillas <- sample(primos)[seq( PARAM$train_final$APO*PARAM$train_final$ksemillerio )]
PARAM$train_final$semillas

[1] 691489

In [ ]:
campos_buenos <- setdiff(
  colnames(dataset_train_final),
  c( "clase_ternaria", "clase01", "training", "azar")
)

In [ ]:
# dejo los datos en formato LightGBM
dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

filas 237073 columnas 925 


[1] "2025-11-16 05:23:30 UTC"

In [ ]:
# genero los modelitos
dir.create( "modelitos", showWarnings= FALSE)

param_completo <- copy( PARAM$train_final$lgbm$param_completo)

for( sem in PARAM$train_final$semillas ) {

  arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
  if( !file.exists( arch_modelo ) )
  {
    param_completo$seed <- sem

    modelito <- lgb.train(
      data= dtrain_final,
      param= param_completo
    )

    lgb.save( modelito, filename= arch_modelo)
    rm(modelito)
    gc()
  }
}

Sys.time()

[1] "2025-11-16 05:46:09 UTC"

### Scoring

Se hace el predict() del modelo en los datos del futuro

In [ ]:
dfuture <- dataset[foto_mes %in% PARAM$train_final$future ]

cols0 <- copy(colnames(dfuture))
filas <- nrow(dfuture)

if( PARAM$train_final$lgbm$qcanaritos > 0 ) {
  for( i in seq(PARAM$train_final$lgbm$qcanaritos) ){
    dfuture[, paste0("canarito_",i) := runif( filas) ]
  }

  # las columnas canaritos mandatoriamente van al comienzo del dataset
  cols_canaritos <- copy( setdiff( colnames(dfuture), cols0 ) )
  setcolorder( dfuture, c( cols_canaritos, cols0 ) )
}

In [ ]:
# dataset de future, donde en este caso estoy haciendo testing

mfuture <- data.matrix(dfuture[, campos_buenos, with= FALSE])

dfuture[, ganancia := ifelse(clase_ternaria=="BAJA+2", 780000, -20000)]

In [ ]:
mganancias <- matrix( nrow=PARAM$train_final$APO, ncol= length(PARAM$train_final$cortes) )

if( file.exists("prediccion.txt") )
  file.remove("prediccion.txt")

In [ ]:
# aplico el modelo a los datos del future

for( vapo in seq(PARAM$train_final$APO) ) {
  # inicializacion en CERO
  vpred_acum <- rep(0.0, nrow(dfuture))
  qacumulados <- 0

  desde <- 1 + (vapo-1)*PARAM$train_final$ksemillerio
  hasta <- desde + PARAM$train_final$ksemillerio - 1
  semillas <- PARAM$train_final$semillas[desde:hasta]

  for( sem in semillas ) {

    arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
    if( file.exists( arch_modelo ) )
    {
      modelo_final <- lgb.load(arch_modelo) # leo del disco
      # hago el predict() y acumulo
      vpred_acum <- vpred_acum + predict(modelo_final, mfuture)
      qacumulados <- qacumulados + 1
      rm(modelo_final)
      gc()
    }
  }

  if( qacumulados > 0 ) {
    vpred_acum <- vpred_acum / qacumulados  # paso a probabildiad
    # tabla de prediccion, puede ser util para futuros ensembles
    #  ya que le modelo ganador va a ser un ensemble de LightGBMs

    tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes, ganancia)]
    tb_prediccion[, meta_modelo := vapo]
    tb_prediccion[, prob := vpred_acum ]
    setorder( tb_prediccion, -prob )
    tb_prediccion[, gan_acum := cumsum(ganancia)]
    tb_prediccion[, ganancia := NULL ]

    # acumulo las ganancias
    for( icor in seq(length(PARAM$train_final$cortes)) ){
      mganancias[ vapo, icor ] <- tb_prediccion[ PARAM$train_final$cortes[icor], gan_acum ]
    }

    # grabo las probabilidades del modelo
    fwrite(tb_prediccion,
      file= "prediccion.txt",
      sep= "\t",
      append= TRUE
    )

    rm(tb_prediccion)
    gc()
  }
}

Sys.time()

[1] "2025-11-16 05:46:22 UTC"

In [ ]:
mganancias

400800000,4.06e+08,4.08e+08,412400000,4.16e+08,413200000,412800000,4.1e+08,4.08e+08


### Clasificacion

Se tomó la decisión de enviar a los 11000 registros con mayor probabilidad de POS={"BAJA+1","BAJA+"}
<br> esto se determinó en forma artesanal analizando meses anterior
<br> esta es una muy importante decisión

In [ ]:
# genero archivos con los  "envios" mejores
dir.create("kaggle", showWarnings=FALSE)

tb_prediccion <- fread("prediccion.txt")

In [ ]:
# genero archivos de fantasia, que NO son el que voy a subir a la Pseudo Competencia Kaggle
envios <- 10500

for( vapo in seq(PARAM$train_final$APO) ) {
  if( tb_prediccion[meta_modelo==vapo, .N] > 0 ) {
    tb_pred <- tb_prediccion[meta_modelo==vapo]
    setorder( tb_pred, -prob )
    tb_pred[, Predicted := 0L] # seteo inicial a 0
    tb_pred[1:envios, Predicted := 1L] # marco los primeros

    archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", vapo, "_", envios, ".csv")

    # grabo el archivo
    fwrite(tb_pred[, list(numero_de_cliente, Predicted)],
      file= archivo_kaggle,
      sep= ","
    )

    rm( tb_pred )
    gc()
  }
}

Sys.time()

[1] "2025-11-16 05:46:22 UTC"

### Subida a Pseudo Kaggle

Aqui viene la verdadera magia de  APO = A Prueba Overfiteres
<br>Se sube un submit a la Pseudo Competencia Kaggle cuya ganancia coincide casi exactamente con la ganancia MEDIA , promediada  APO veces

In [ ]:
#colmedias <- colMeans( mganancias, na.rm=TRUE )
#mcorte_mejor <- max(colmedias, na.rm=TRUE)
#icorte_mejor <- which.max( colmedias )
#corte_mejor <- PARAM$train_final$cortes[icorte_mejor]

In [ ]:
#tbl <- as.data.table( as.list( colmedias ) )
#colnames(tbl) <- paste0( "e", PARAM$train_final$cortes )
#tbl[, experimento := PARAM$experimento ]
#
#exp_gral <- "/content/buckets/b1/exp/apo-gral"
#dir.create(exp_gral, showWarnings=FALSE)
#fwrite( tbl,
#  file= paste0( exp_gral, "/tb_experimentos.txt"),
#  sep= "\t",
#  append= TRUE
#)

In [ ]:
#colnames( mganancias ) <- paste0( "e", PARAM$train_final$cortes )
#tbl_local <- as.data.table( mganancias )
#
#fwrite( tbl_local,
#  file= "tb_apo.txt",
#  sep= "\t"
#)

In [ ]:
#icerca <- which.min(  abs( tb_prediccion$gan_acum - mcorte_mejor ) )
#vmodelo <- tb_prediccion[ icerca, meta_modelo ]
#tb_pred <- tb_prediccion[meta_modelo==vmodelo]

#mcorte_mejor
#icerca
#tb_prediccion[ icerca]

[1] 4.16e+08

[1] 10000

numero_de_cliente,foto_mes,meta_modelo,prob,gan_acum
<int>,<int>,<int>,<dbl>,<dbl>
938889584,202106,1,0.4308785,4.16e+08


In [ ]:
#icerca <- which.min(  abs( tb_pred$gan_acum - mcorte_mejor ) )
#icerca

[1] 10000

In [ ]:
#icerca <- which.min(  abs( tb_prediccion$gan_acum - mcorte_mejor ) )
#vmodelo <- tb_prediccion[ icerca, meta_modelo ]
#tb_pred <- tb_prediccion[meta_modelo==vmodelo]

#icerca <- which.min(  abs( tb_pred$gan_acum - mcorte_mejor ) )
#tb_pred[, Predicted := 0L] # seteo inicial a 0
#tb_pred[1:icerca, Predicted := 1L] # marco los primeros

#archivo_pseudo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_",  icerca, ".csv")

# grabo el archivo
#fwrite(tb_pred[, list(numero_de_cliente, Predicted)],
#  file= archivo_pseudo_kaggle,
#  sep= ","
#)


In [ ]:
# la subida a Kaggle
#comando <- "kaggle competitions submit"
#competencia <- "-c  test-202106"
#arch <- paste( "-f", archivo_pseudo_kaggle)
#mensaje <-  paste0( "-m 'exp=", PARAM$experimento,
#  "  ", paste(names(plocal), plocal, sep= "=", collapse= ";" ),
#  " envios=", icorte_mejor,"'")


#linea <- paste( comando, competencia, arch, mensaje)
#salida <- system(linea, intern=TRUE)
#cat(salida)

Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c  test-202106 -f ./kaggle/KAComp2_Prueba-002_10000.csv -m 'exp=Comp2_Prueba-002  qcanaritos=5;min_data_in_leaf=2000;learning_rate=1;gradient_bound=0.01;APO=1;ksemillerio=1 envios=5'' had status 1”


In [ ]:
Sys.time()

[1] "2025-11-16 05:46:23 UTC"